In [11]:
import numpy as np
import numpy.linalg as lin

In [12]:
def getZ(label:str) -> int:    
    elements="H   He\
        Li  Be  B   C   N   O   F   Ne\
        Na  Mg  Al  Si  P   S   Cl  Ar\
        K   Ca  Sc  Ti  V   Cr  Mn  Fe  Co  Ni  Cu  Zn  Ga  Ge  As  Se  Br  Kr\
        Rb  Sr  Y   Zr  Nb  Mo  Tc  Ru  Rh  Pd  Ag  Cd  In  Sn  Sb  Te  I   Xe\
        Cs  Ba  La  Ce  Pr  Nd  Pm  Sm  Eu  Gd  Tb  Dy  Ho  Er  Tm  Yb\
        Lu  Hf  Ta  W   Re  Os  Ir  Pt  Au  Hg  Tl  Pb  Bi  Po  At  Rn\
        Fr  Ra  Ac  Th  Pa  U".split()    
    
    return elements.index(label)+1

In [13]:
def importQM7(structure_file:str, energy_file:str):
    """
    Return: Z, R, E\n
    Z: list of 1D-arrays containing atomic identities\n
    R: list of 2D-arrays containing atomic positions\n
    E: 1D-array containing atomization energy\n
    """
    structures = open(structure_file,  'r').readlines()

    Z = []
    R = []
    E = []
    n_max = 0

    for line in range(len(structures)):
        x = structures[line].split()

        #Check for start of molecule structure data:
        if len(x) == 1:
            n_atoms = int(x[0])
            if n_atoms > n_max: n_max = n_atoms

            Zs   = np.zeros(n_atoms)
            xyzs = np.zeros((n_atoms, 3))

            #Go through every atom in the molecule:
            atom_index = 0
            for j in range(line+2, line+2+n_atoms):
                Zs  [atom_index] = getZ(structures[j].split()[0])
                xyzs[atom_index] = np.array([float(val) for val in structures[j].split()[1:]])

                atom_index += 1
            
            Z.append(Zs)
            R.append(xyzs)
        
    file = open(energy_file,  'r').readlines()
    for line in range(len(file)):
        E.append(float(file[line].split()[0]))
    
    return Z, R, E

In [14]:
def coulomb_eigenvalues(Z, R, n_max, outputname:str):
    n_mols = len(Z)
    n_max  = n_max
    #Generate Descriptors, eigenvalues of Coulomb Matrix M
    coulomb_eVs = np.zeros((n_mols, n_max))
    
    for k in range(n_mols):
        n_atoms = len(Z[k])

        M = np.zeros((n_atoms, n_atoms))

        for i in range(n_atoms):
            for j in range(n_atoms):
                if i == j:
                    M[i][j] = 0.5 * (Z[k][i])**2.4
                else:
                    M[i][j] = (Z[k][i]*Z[k][j]) / lin.norm(R[k][i] - R[k][j])**2

        eigenValues = lin.eigvals(M)
        sorted_eVal = np.array(sorted(eigenValues, key=abs, reverse=True))

        #Append 0s to match molecule with largest number of eigenvalues
        if n_atoms == n_max:
            coulomb_eVs[k] = sorted_eVal
        else:
            coulomb_eVs[k] = np.concatenate((sorted_eVal, [0]*(n_max-n_atoms)))

    np.savetxt(fname=outputname, X=coulomb_eVs, delimiter=" ", newline="\n")
    return coulomb_eVs

In [15]:
def stratSplit(nData:int, nTrain:int, k:int, y):
    assert nData  > nTrain
    assert nData == len(y)

    sortedIndex = np.argsort(y)
    strata = [[] for _ in range(k)]

    max_strata = nTrain // k
    
    for i in range(k):
        sample = i
        selected = 0 
        
        while selected < nData:
            if sample >= nData: break
            strata[i].append(sortedIndex[sample])
            sample   += k
            selected += 1

    for i in range(len(strata)):
        np.random.shuffle(strata[i])
    
    trainingIndex = np.array([])
    testingIndex  = np.array([])

    for i in range(k):
        per_strata = max_strata

        if len(trainingIndex) + max_strata > nTrain:
            per_strata = nTrain - len(trainingIndex)

        if (i == k-1) and (len(trainingIndex) + per_strata < nTrain):
            per_strata = nTrain - len(trainingIndex)
            
        trainingIndex = np.concatenate((trainingIndex, strata[i][:per_strata]), 
                                        casting="unsafe", dtype=int)
        testingIndex  = np.concatenate((testingIndex,  strata[i][per_strata:]), 
                                        casting="unsafe", dtype=int)

    assert len(trainingIndex) == nTrain        , \
        f"Unexpected array size. {len(trainingIndex)} != {nTrain}"
        
    assert len(testingIndex ) == nData - nTrain, \
        f"Unexpected array size. {len(trainingIndex)} != {nData - nTrain}"
    
    return trainingIndex, testingIndex

In [16]:
model_size = 1000

Z_small, R_small, E_small = importQM7(structure_file = "sizeSplit/qm7_small.txt", 
                                      energy_file    = "sizeSplit/PBE0_small.txt")
Z_rest,  R_rest,  E_rest  = importQM7(structure_file = "sizeSplit/qm7_rest.txt", 
                                      energy_file    = "sizeSplit/PBE0_rest.txt")

strat_train, strat_test = stratSplit(nData  = len(Z_rest),
                                     nTrain = model_size - len(Z_small),
                                     k = 5,
                                     y = E_rest)

Z_train, R_train, E_train = Z_small, R_small, E_small
Z_test , R_test , E_test  = [], [], []


for i in range(len(strat_train)):
    Z_train.append(Z_rest[strat_train[i]])
    R_train.append(R_rest[strat_train[i]])
    E_train.append(E_rest[strat_train[i]])

for i in range(len(strat_test)):
    Z_test.append(Z_rest[strat_test[i]])
    R_test.append(R_rest[strat_test[i]])
    E_test.append(E_rest[strat_test[i]])


if (len(max(Z_train, key=len)) != len(max(Z_test, key=len))):
    print("Mismatch in maximum molecule size between training and testing sets. Be cautious.")

n_max = max(len(max(Z_train, key=len)),
            len(max(Z_test, key=len)))

In [17]:
training_data = coulomb_eigenvalues(Z=Z_train, R=R_train, n_max=n_max, outputname="trainingSplit/coulomb_train.txt")
training_trgt = np.array(E_train)
np.savetxt(fname="trainingSplit/PBE0_train.txt", X=training_trgt)


testing_data  = coulomb_eigenvalues(Z=Z_test, R=R_test, n_max=n_max, outputname="trainingSplit/coulomb_test.txt")
testing_trgt  = np.array(E_test)
np.savetxt(fname="trainingSplit/PBE0_test.txt", X=testing_trgt)